# Assignment 2

In this assigment, we will work with the *Adult* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Extract the data files into the subdirectory: `../05_src/data/adult/` (relative to `./05_src/`).

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../05_src/data/adult/`, then you can use the code below to load them.

In [1]:
import pandas as pd
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]
adult_dt = (pd.read_csv('../../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))


# Get X and Y

Create the features data frame and target data:

+ Create a dataframe `X` that holds the features (all columns that are not `income`).
+ Create a dataframe `Y` that holds the target data (`income`).
+ From `X` and `Y`, obtain the training and testing data sets:

    - Use a train-test split of 70-30%. 
    - Set the random state of the splitting function to 42.

In [2]:
# Create features and target data
X = adult_dt.drop(columns='income')
Y = adult_dt['income']

In [3]:
from sklearn.model_selection import train_test_split

# Split the data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)


## Random States

Please comment: 

+ What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? 
+ Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)?

*(Comment here.)*

1) The random state is a seed value used by the random number generator. In this case, it's set to 42.

2) It is useful because it ensures reproducibility of the results. By setting a random state, the same data split can be achieved every time the code is run, making it easier to compare results across different runs.

# Preprocessing

Create a [Column Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) that treats the features as follows:

- Numerical variables

    * Apply [KNN-based imputation for completing missing values](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html):
        
        + Consider the 7 nearest neighbours.
        + Weight each neighbour by the inverse of its distance, causing closer neigbours to have more influence than more distant ones.
    * [Scale features using statistics that are robust to outliers](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

- Categorical variables: 
    
    * Apply a [simple imputation strategy](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer):

        + Use the most frequent value to complete missing values, also called the *mode*.

    * Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):
        
        + Handle unknown labels if they exist.
        + Drop one column for binary variables.
    
    
The column transformer should look like this:

![](./images/assignment_2__column_transformer.png)

In [6]:

from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder

num_features = X.select_dtypes(include=['int64', 'float64']).columns
cat_features = X.select_dtypes(include=['object']).columns

num_transformer = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=7, weights='distance')),
    ('scaler', RobustScaler())
])

cat_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', drop='if_binary'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, num_features),
        ('cat', cat_transformer, cat_features)
    ]
)



## Model Pipeline

Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `classifier` and assign a [`RandomForestClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to it.

The pipeline looks like this:

![](./images/assignment_2__pipeline.png)

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])



# Cross-Validation

Evaluate the model pipeline using [`cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

+ Measure the following [preformance metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values): negative log loss, ROC AUC, accuracy, and balanced accuracy.
+ Report the training and validation results. 
+ Use five folds.


In [14]:
scoring = {
    'neg_log_loss': make_scorer(log_loss, greater_is_better=False, needs_proba=True),
    'roc_auc': make_scorer(roc_auc_score, needs_proba=True),
    'accuracy': make_scorer(accuracy_score),
    'balanced_accuracy': make_scorer(balanced_accuracy_score)
}

cv_results = cross_validate(pipeline, X_train, y_train, cv=5, scoring=scoring, return_train_score=True)

# Print Results
print("Training Results:")
print(f"Negative Log Loss: {cv_results['train_neg_log_loss'].mean()}")
print(f"ROC AUC: {cv_results['train_roc_auc'].mean()}")
print(f"Accuracy: {cv_results['train_accuracy'].mean()}")
print(f"Balanced Accuracy: {cv_results['train_balanced_accuracy'].mean()}")
print("\nValidation Results:")
print(f"Negative Log Loss: {cv_results['test_neg_log_loss'].mean()}")
print(f"ROC AUC: {cv_results['test_roc_auc'].mean()}")
print(f"Accuracy: {cv_results['test_accuracy'].mean()}")
print(f"Balanced Accuracy: {cv_results['test_balanced_accuracy'].mean()}")





/Users/mishavitvinov/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/metrics/_scorer.py:548: FutureWarning: The `needs_threshold` and `needs_proba` parameter are deprecated in version 1.4 and will be removed in 1.6. You can either let `response_method` be `None` or set it to `predict` to preserve the same behaviour.
  warnings.warn(


ValueError: Cannot have number of splits n_splits=5 greater than the number of samples: n_samples=2.

Display the fold-level results as a pandas data frame and sorted by negative log loss of the test (validation) set.

In [15]:
cv_results_df = pd.DataFrame(cv_results)
cv_results_df = cv_results_df.sort_values(by='test_neg_log_loss', ascending=False)
print(cv_results_df)

   fit_time  score_time  test_neg_log_loss  train_neg_log_loss  test_roc_auc  \
0  3.503534    0.130449          -0.350084           -0.081227      0.904881   
3  3.516906    0.121852          -0.356626           -0.082628      0.907304   
1  3.508265    0.125981          -0.362267           -0.081582      0.901218   
2  3.532977    0.121885          -0.376556           -0.081577      0.900899   
4  3.458558    0.121976          -0.388065           -0.081301      0.902014   

   train_roc_auc  test_accuracy  train_accuracy  test_balanced_accuracy  \
0            1.0       0.853257        0.999945                0.778373   
3            1.0       0.860026        1.000000                0.781773   
1            1.0       0.846896        1.000000                0.767721   
2            1.0       0.852348        0.999945                0.775166   
4            1.0       0.856516        1.000000                0.775763   

   train_balanced_accuracy  
0                 0.999887  
3         

Calculate the mean of each metric. 

In [16]:
mean_results = cv_results_df.mean()
print(mean_results)

fit_time                   3.504048
score_time                 0.124429
test_neg_log_loss         -0.366720
train_neg_log_loss        -0.081663
test_roc_auc               0.903263
train_roc_auc              1.000000
test_accuracy              0.853809
train_accuracy             0.999978
test_balanced_accuracy     0.775759
train_balanced_accuracy    0.999955
dtype: float64


Calculate the same performance metrics (negative log loss, ROC AUC, accuracy, and balanced accuracy) using the testing data `X_test` and `Y_test`. Display results as a dictionary.

*Tip*: both, `roc_auc()` and `neg_log_loss()` will require prediction scores from `pipe.predict_proba()`. However, for `roc_auc()` you should only pass the last column `Y_pred_proba[:, 1]`. Use `Y_pred_proba` with `neg_log_loss()`.

In [17]:
from sklearn.metrics import roc_auc_score, log_loss, accuracy_score, balanced_accuracy_score

pipeline.fit(X_train, y_train)
y_pred_proba = pipeline.predict_proba(X_test)

metrics = {
    'neg_log_loss': log_loss(y_test, y_pred_proba),
    'roc_auc': roc_auc_score(y_test, y_pred_proba[:, 1]),
    'accuracy': accuracy_score(y_test, pipeline.predict(X_test)),
    'balanced_accuracy': balanced_accuracy_score(y_test, pipeline.predict(X_test))
}

print(metrics)


{'neg_log_loss': 0.40069803789358494, 'roc_auc': 0.8997319555477492, 'accuracy': 0.8543351417750026, 'balanced_accuracy': 0.774775620012208}


# Target Recoding

In the first code chunk of this document, we loaded the data and immediately recoded the target variable `income`. Why is this [convenient](https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case)?

The specific line was:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```

(Answer here.)

Recoding the target variable income during the data loading step is convenient because it simplifies the preprocessing pipeline. It ensures that the target variable is binary and numerical, which is necessary for many machine learning algorithms and metrics. This transformation can be seamlessly integrated into the data loading process, ensuring that the data is ready for model training and evaluation without additional steps.


## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Evaluation of model pipeline |Model pipeline was evaluated correctly.|Model pipeline was not evaluated correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.